In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [3]:
!pip install gdown
import gdown

file_id = '1-BKlJlBt30s3uzGFHGtrKSj-iXsdOIID'
output_path = '/content/blood_model_improved3.h5'
gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1-BKlJlBt30s3uzGFHGtrKSj-iXsdOIID
From (redirected): https://drive.google.com/uc?id=1-BKlJlBt30s3uzGFHGtrKSj-iXsdOIID&confirm=t&uuid=314956e4-5291-4169-9613-d068e6e7f362
To: /content/blood_model_improved3.h5
100%|██████████| 189M/189M [00:01<00:00, 169MB/s]


'/content/blood_model_improved3.h5'

In [4]:
import os
import numpy as np
from PIL import Image
import gradio as gr
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load trained model once
model = load_model('/content/blood_model_improved3.h5')
image_size = (256, 256)

def process_and_predict(uploaded_image):
    # Save original uploaded image
    img = Image.open(uploaded_image).convert('RGB')
    width, height = img.size
    part_width = width // 3

    # Step 1: Crop into 3 parts
    parts = []
    for i in range(3):
        left = i * part_width
        right = (i + 1) * part_width if i < 2 else width
        cropped = img.crop((left, 0, right, height))
        part_path = f"/content/red_dot_part_{i+1}.png"
        cropped.save(part_path)
        parts.append(part_path)

    # Step 2: Resize and predict each part
    predictions = []
    for part in parts:
        img_part = Image.open(part).resize(image_size)
        img_array = image.img_to_array(img_part) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array, verbose=0)[0][0]
        predictions.append(1 if pred > 0.5 else 0)

    # Step 3: Rule-based blood type prediction
    first, second, third = predictions
    if first and second:
        result = "AB"
    elif first:
        result = "A"
    elif second:
        result = "B"
    else:
        result = "O"
    result += " Positive" if third else " Negative"

    # Step 4: Show cropped images with labels
    fig, axs = plt.subplots(1, 3, figsize=(10, 4))
    for i, part in enumerate(parts):
        axs[i].imshow(Image.open(part))
        axs[i].set_title(f"Part {i+1}: {'Positive' if predictions[i] else 'Negative'}")
        axs[i].axis('off')
    plt.tight_layout()
    fig_path = "/content/prediction_visual.png"
    plt.savefig(fig_path)
    plt.close()

    return result, fig_path

# Gradio interface
interface = gr.Interface(
    fn=process_and_predict,
    inputs=gr.Image(type="filepath", label="Upload Blood Dot Image"),
    outputs=[
        gr.Textbox(label="Predicted Blood Type"),
        gr.Image(label="Split Parts with Predictions")
    ],
    title="🩸 Blood Group and Rh Detection",
    description="Upload a single image of the blood test strip. The app splits it into 3 parts, predicts each using a trained CNN, and returns the final blood type."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db8ef102902d4d8bd2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
